# APP Evaluation

In [7]:
import pandas as pd
from pathlib import Path
from config import *

from utils import rsquareCI, rsquareCI_v2, R2difference

In [8]:
# load Phone data
Ummae = pd.read_feather("Raw/Ummae.feather").query("DataSetName == 'APP-EV'")
Ummae

,eid,type,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,...,TSH,UA,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age
71337,00167ec66f147def3fe46a8c53dcbb4d,None,APP-EV,16.022173,78.338066,21.357365,31.359474,19.945280,45.230553,26.974274,...,1.966572,339.672302,6.326898,6.865973,77.575577,1.561841,31.097942,3.012093,0.651726,74.216469
71338,fd29850b22d405ba1b1061db5f334f67,None,APP-EV,15.547345,79.268669,21.030893,31.297989,19.934504,44.367851,26.182653,...,1.941188,346.592773,6.450529,6.242180,79.093575,1.598619,30.850290,2.851500,0.632225,68.109055
71339,2dd42e3e3fff0d9f846f6c130b211b2e,None,APP-EV,16.050095,86.790749,19.116692,31.370438,20.074612,46.133671,27.554487,...,2.006614,294.745300,5.713026,6.551788,67.441803,1.844779,31.659286,3.162086,0.741407,63.587364
71340,cd0ba672d1d4aa1f29a0d6a4d205da7f,None,APP-EV,15.951509,80.521355,20.053829,31.194374,19.933733,45.672615,27.685871,...,1.939758,361.789917,6.145970,6.600042,75.635201,1.763962,31.279552,3.138473,0.702724,67.691010
71341,b4ec8b6e3c8111634cda882262affe44,None,APP-EV,15.845858,76.594109,20.532967,31.284309,19.896782,44.375935,28.010660,...,1.927603,366.471008,6.354002,6.795804,83.876236,1.565067,30.932840,3.214008,0.746629,66.926979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71442,6a21820a6e2b3d28535eb1b01b46b640,None,APP-EV,15.989254,82.741898,15.321168,30.421919,19.902170,44.787785,25.803036,...,1.948339,315.635193,6.220623,6.351908,64.440727,1.765253,32.171097,3.144820,0.742420,74.552834
71443,937dd8fba9431798b246bcf19cfa16b9,None,APP-EV,15.933067,77.426376,24.055159,31.480536,19.958368,44.940968,30.713423,...,1.927245,349.348907,6.070603,6.957904,82.771080,1.913941,30.982370,3.202075,0.751433,54.688137
71444,bf757af582f5b6d44d6a3ba96e5216cf,None,APP-EV,15.877915,84.254593,17.153166,30.718622,19.925266,44.793846,23.754135,...,2.037718,280.089539,5.695670,6.077231,59.378605,1.754768,31.824387,3.234574,0.751625,61.129505
71445,137d4f7bc61f0cbafc7b20c7f4821e40,None,APP-EV,15.906353,75.265579,22.995594,31.540623,19.988390,45.784470,25.238533,...,1.969432,364.983551,5.908635,6.424060,72.195877,1.431825,31.213512,2.845407,0.664325,52.362419


In [9]:
# Load clinical label
label_df = pd.read_feather("Raw/ClinicalLabels.feather").query(
    "DataSetName == 'APP-EV'"
)
label_df

,eid,DataSetName,AG,ALP,ALT,APTT,AST,Alb,BMI,BNP,...,Urea,WBC,Weight,hs-CRP,hsTnI,nonHDL,sdLDL,Age,Sex,Stenosis
9660,e9dd175c8b07cb86498d00b21377e0f1,APP-EV,2.02,54.9,45.0,24.77,21.0,46.7,27.281746,NaN,...,12.37,3.28,77.0,2.1,2.4,1.73,0.46,55.0,Male,NaN
9661,942c57084be3dbfadd54bf49f681cc15,APP-EV,1.56,NaN,40.3,24.37,27.4,NaN,24.111507,NaN,...,5.14,4.58,73.0,0.8,3.7,2.06,NaN,60.0,Male,NaN
9662,2f6f6be7939d39843b9a317c9e131b1e,APP-EV,1.53,NaN,40.7,25.96,23.4,NaN,27.064052,NaN,...,5.47,6.04,81.0,NaN,2.4,2.05,NaN,61.0,Male,NaN
9663,75c8fe9721a3f400103f2c1896d43e6b,APP-EV,1.86,60.0,17.7,24.66,18.7,44.7,24.441802,NaN,...,5.15,5.21,74.0,0.7,2.7,1.20,0.34,67.0,Male,NaN
9664,629a24178a9545f25d5fa73ac9b80013,APP-EV,1.49,NaN,25.0,27.97,19.6,NaN,25.909457,NaN,...,5.91,6.69,74.0,NaN,2.8,1.86,NaN,60.0,Male,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,1f5b5321a421d4a566f6c41262ba774e,APP-EV,1.43,95.4,15.3,NaN,18.6,42.4,19.948060,NaN,...,4.77,NaN,53.0,0.3,NaN,1.72,0.41,60.0,Female,NaN
9766,234d3b32bb21791791da8741b6dd072b,APP-EV,0.98,100.5,23.3,NaN,28.8,NaN,27.942143,NaN,...,5.99,NaN,68.0,5.4,NaN,0.86,0.17,72.0,Female,NaN
9767,9b83d1551a89149543cdd635f8f2690c,APP-EV,1.38,NaN,14.9,NaN,21.9,NaN,19.628265,NaN,...,5.36,NaN,49.0,0.3,NaN,2.73,NaN,59.0,Female,NaN
9768,e0b7bca81e873378e05ddf98f369d4ed,APP-EV,1.53,NaN,28.9,NaN,26.2,NaN,32.466181,NaN,...,5.43,NaN,78.0,4.6,NaN,2.68,NaN,69.0,Female,NaN


## R2

In [4]:
Ummae[["eid"]]

,eid
71337,00167ec66f147def3fe46a8c53dcbb4d
71338,fd29850b22d405ba1b1061db5f334f67
71339,2dd42e3e3fff0d9f846f6c130b211b2e
71340,cd0ba672d1d4aa1f29a0d6a4d205da7f
71341,b4ec8b6e3c8111634cda882262affe44
...,...
71442,6a21820a6e2b3d28535eb1b01b46b640
71443,937dd8fba9431798b246bcf19cfa16b9
71444,bf757af582f5b6d44d6a3ba96e5216cf
71445,137d4f7bc61f0cbafc7b20c7f4821e40


In [5]:
# Calculate R2 for all traits
from ppp_prediction.corr_v3 import cal_corr_v3

compare_res_list = []
data_phone_res_dict = {}
errors = []
for trait in qts:
    # for trait in ['HDLC']:
    if trait not in label_df.columns:
        errors.append(trait)
        continue
    # for trait in ['HDLC']:
    try:
        df = (
            label_df[["eid", trait]]
            .merge(
                Ummae[["eid", trait]],
                on="eid",
                how="inner",
                suffixes=("_lab", "_ummae"),
            )
            .dropna()
            .groupby("eid")
            .mean()
        )  # get mean of ummae
    except:
        errors.append(trait)
        continue

    # merge prs
    # break
    df = df.dropna(axis=1, how="all")
    print(df.shape)

    metrics_qt = cal_corr_v3(
        df,
        x=f"{trait}_ummae",
        y=f"{trait}_lab",
        norm_x=True,
        # cov = cov
        model_type="glm",
    )
    compare_res_list.append(metrics_qt)
    data_phone_res_dict[trait] = df

from ppp_prediction.corr_v3 import generate_multipletests_result

compare_res_df_fdr = (
    generate_multipletests_result(
        pd.concat(compare_res_list).dropna(subset="pvalue"), method="fdr_bh"
    )
    .sort_values("r2_score", ascending=False)[
        ["x", "y", "pearsonr", "spearmanr", "N", "r2_score", "p_adj", "pvalue"]
    ]
    .rename(columns={"r2_score": "R2"})
)

compare_res_df_fdr = compare_res_df_fdr.rename(columns={"x": "Feature", "y": "Traits"})
compare_res_df_fdr["Traits"] = compare_res_df_fdr["Traits"].str.split("_").str[0]
compare_res_df_fdr["Feature"] = compare_res_df_fdr["Feature"].apply(
    lambda x: "Face" if "ummae" in x else x
)

from ppp_prediction.metrics.utils import format_metrics

se, lci, uci = rsquareCI_v2(compare_res_df_fdr["R2"], compare_res_df_fdr["N"], 1)
compare_res_df_fdr["R2_SE"] = se
compare_res_df_fdr["R2_LCI"] = lci
compare_res_df_fdr["R2_UCI"] = uci

add_metric = ["R2"]

for metric in add_metric:

    compare_res_df_fdr[f"{metric} (95% CI)"] = format_metrics(
        compare_res_df_fdr[f"{metric}"],
        x_lci=compare_res_df_fdr[f"{metric}_LCI"],
        x_uci=compare_res_df_fdr[f"{metric}_UCI"],
        round=3,
        data=compare_res_df_fdr,
    )
compare_res_df_fdr

compare_res_df_fdr.sort_values("R2", ascending=False)

(110, 2)
threads is 4 and x is AG_ummae and y is AG_lab, which is str, so don't supported for multi threads
AG_lab ~ AG_ummae
(54, 2)
threads is 4 and x is ALP_ummae and y is ALP_lab, which is str, so don't supported for multi threads
ALP_lab ~ ALP_ummae
(107, 2)
threads is 4 and x is ALT_ummae and y is ALT_lab, which is str, so don't supported for multi threads
ALT_lab ~ ALT_ummae
(59, 2)
threads is 4 and x is APTT_ummae and y is APTT_lab, which is str, so don't supported for multi threads
APTT_lab ~ APTT_ummae
(108, 2)
threads is 4 and x is AST_ummae and y is AST_lab, which is str, so don't supported for multi threads
AST_lab ~ AST_ummae
(51, 2)
threads is 4 and x is Alb_ummae and y is Alb_lab, which is str, so don't supported for multi threads
Alb_lab ~ Alb_ummae
(106, 2)
threads is 4 and x is BMI_ummae and y is BMI_lab, which is str, so don't supported for multi threads
BMI_lab ~ BMI_ummae
(0, 0)
threads is 4 and x is BNP_ummae and y is BNP_lab, which is str, so don't supported for

,Feature,Traits,pearsonr,spearmanr,N,R2,p_adj,pvalue,R2_SE,R2_LCI,R2_UCI,R2 (95% CI)
0,Face,Age,0.886285,0.872709,110.0,0.785501,2.998469e-86,5.260472e-88,0.035119,0.715263,0.855738,"0.786 (0.715, 0.856)"
0,Face,Height,0.718001,0.670828,104.0,0.515526,5.840332e-24,2.049239e-25,0.065966,0.383594,0.647458,"0.516 (0.384, 0.647)"
0,Face,Weight,0.708498,0.698309,103.0,0.501970,1.167628e-22,6.145412e-24,0.067217,0.367537,0.636403,"0.502 (0.368, 0.636)"
0,Face,BMI,0.617861,0.607347,106.0,0.381753,1.587715e-14,1.114186e-15,0.071799,0.238155,0.525350,"0.382 (0.238, 0.525)"
0,Face,RBC,0.477506,0.491986,57.0,0.228012,3.966344e-04,5.566799e-05,0.091854,0.044304,0.411720,"0.228 (0.044, 0.412)"
0,Face,Cr,0.475419,0.388958,107.0,0.226024,3.499273e-07,3.069538e-08,0.068859,0.088305,0.363742,"0.226 (0.088, 0.364)"
0,Face,ALP,0.469732,0.462591,54.0,0.220649,7.889712e-04,1.245744e-04,0.093403,0.033843,0.407454,"0.221 (0.034, 0.407)"
0,Face,HDLC,0.423971,0.418713,107.0,0.179752,1.531001e-05,1.611580e-06,0.065079,0.049594,0.309910,"0.180 (0.050, 0.310)"
0,Face,Hb,0.416151,0.379363,55.0,0.173182,4.917587e-03,8.627346e-04,0.087089,-0.000996,0.347360,"0.173 (-0.001, 0.347)"
0,Face,eGFR(CKD-EPI),0.374358,0.450339,105.0,0.140144,3.404375e-04,4.180811e-05,0.060771,0.018601,0.261687,"0.140 (0.019, 0.262)"


In [6]:
compare_res_df_fdr.reset_index(drop=True).to_feather(RawDataDir / "Phone_qt_df.feather")

## ICC

In [10]:
other = pd.read_feather(RawDataDir / "metrics_qt.feather").query("Model == 'Ummae'")[
    ["Label", "Center", "R2"]
]

In [11]:
Other_AZ_GEV = (
    pd.read_feather(RawDataDir / "Genetics_120_results.feather")
    .query("Feature == 'Face'")
    .rename(columns={"Traits": "Label"})
    .assign(Center="AZ-GEV")[["Label", "Center", "R2"]]
)
Other_AZ_GEV

,Label,Center,R2
0,Weight,AZ-GEV,0.503524
1,Height,AZ-GEV,0.464711
2,HCT,AZ-GEV,0.409777
3,Hb,AZ-GEV,0.409413
4,BMI,AZ-GEV,0.353933
5,HDLC,AZ-GEV,0.319215
6,Cr,AZ-GEV,0.243655
8,eGFR(CKD-EPI),AZ-GEV,0.188642
17,UA,AZ-GEV,0.164182
53,GGT,AZ-GEV,0.125439


In [13]:
other = pd.concat([other, Other_AZ_GEV])
by_all_df = pd.concat(
    [
        other,
        compare_res_df_fdr[["Traits", "R2"]]
        .rename(columns={"Traits": "Label"})
        .assign(Center="Phone"),
    ]
)
by_all_df

,Label,Center,R2
122,AG,AZ-EV1,0.001845
123,ALP,AZ-EV1,0.025919
124,ALT,AZ-EV1,0.039292
125,APTT,AZ-EV1,0.016927
126,AST,AZ-EV1,0.000225
...,...,...,...
0,TP,Phone,0.002142
0,TSH,Phone,0.001235
0,AG,Phone,0.000943
0,T4,Phone,0.000149


In [16]:
from pingouin import intraclass_corr

ICC_df_list = []
for check_center in [
    "AZ-TR",
    "AZ-EV1",
    "AZ-EV2",
    "DX-EV",
    "AZTZ-EV",
    # "AZ-GEV"
]:
    by_all_df_melt = by_all_df.query("Center == @check_center or Center == 'Phone'")
    by_all_df_melt = by_all_df_melt.dropna(how="any")
    ICC_df = (
        intraclass_corr(
            data=by_all_df_melt,
            targets="Label",  # 分组列
            raters="Center",  # 数据来源（区分抽样来源）
            ratings="R2",  # 概率值列
            nan_policy="omit",
        )
        .round(2)
        .query("Type == 'ICC2'")
        .assign(ComapreCenter=check_center)
        # .set_index("Type")
    )
    ICC_df_list.append(ICC_df)

ICC_df_by_center = pd.concat(ICC_df_list)
ICC_df_by_center["ICC_LCI"] = ICC_df_by_center["CI95%"].apply(lambda x: x[0])
ICC_df_by_center["ICC_UCI"] = ICC_df_by_center["CI95%"].apply(lambda x: x[1])
from ppp_prediction.metrics.utils import format_metrics

ICC_df_by_center["ICC 95% CI"] = format_metrics(
    ICC_df_by_center["ICC"],
    x_uci=ICC_df_by_center["ICC_UCI"],
    x_lci=ICC_df_by_center["ICC_LCI"],
)

ICC_df_by_center

,Type,Description,ICC,F,df1,df2,pval,CI95%,ComapreCenter,ICC_LCI,ICC_UCI,ICC 95% CI
1,ICC2,Single random raters,0.88,15.85,56,56,0.0,"[0.81, 0.93]",AZ-TR,0.81,0.93,"0.88 (0.81, 0.93)"
1,ICC2,Single random raters,0.91,20.02,55,55,0.0,"[0.85, 0.94]",AZ-EV1,0.85,0.94,"0.91 (0.85, 0.94)"
1,ICC2,Single random raters,0.94,31.15,55,55,0.0,"[0.9, 0.96]",AZ-EV2,0.90,0.96,"0.94 (0.90, 0.96)"
1,ICC2,Single random raters,0.94,32.16,38,38,0.0,"[0.88, 0.97]",DX-EV,0.88,0.97,"0.94 (0.88, 0.97)"
1,ICC2,Single random raters,0.90,19.46,44,44,0.0,"[0.83, 0.94]",AZTZ-EV,0.83,0.94,"0.90 (0.83, 0.94)"


In [18]:
ICC_df_by_center.reset_index(drop=True).to_feather(
    RawDataDir / "ICC_df_by_center_APP.feather"
)